# Assigment 6

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

1. Generate a folder with the name of your group in this folder **GitHub\Diplomado_PUCP\_output\sbs\municipal**. The code should run even if the folder exists.
2. Scrape all the available information in this [link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C). Save an excel file by year and month request. The excel file should be name using the month and year of the sheet.

### Installing packages

In [ ]:
#!pip install selenium
#!pip install webdriver-manager

### Importing libraries

In [1]:
# This library is to manipulate browser
from selenium import webdriver

# it allows you to work with differen versions of drivers
# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import os

# Options driver
from selenium.webdriver.chrome.options import Options

# For dataframes
import pandas as pd

# For simulate human behavior.
import time
from selenium.webdriver.common.by import By

import numpy as np

### Generating the folder

In [ ]:
folder_name = "group5"
relative_path = "..\..\_output\sbs\municipal/" + folder_name

if not os.path.exists(relative_path):
    os.makedirs(relative_path)
    print(f"Folder '{folder_name}' created successfully!")
else:
    print(f"Folder '{folder_name}' already exists!")

In [ ]:
# install ChromeDriverManager
driver = webdriver.Chrome( ChromeDriverManager().install() )
driver.maximize_window()

# Setting
options= Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)
# we open our page
driver.get("https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C")

### Identifying dropdowns and buttons

In [4]:
# Dropdown button list
dropdown_button_year = driver.find_element( By.ID, "ctl00_cphContent_rAnio")
dropdown_button_year.click()

In [5]:
# Generate the year list from the dropdown
list_year = driver.find_element( By.ID,  "ctl00_cphContent_rAnio_DropDown" )
years_list = list_year.find_elements( By.TAG_NAME, 'li')
list_year_options = len(years_list)

In [6]:
# Dropdown button month
dropdown_button_month = driver.find_element( By.ID,  "ctl00_cphContent_rMes")
dropdown_button_month.click()

In [7]:
# Generate the month list from the dropdown
list_month = driver.find_element( By.ID,  "ctl00_cphContent_rMes_DropDown" )
months_list = list_month.find_elements( By.TAG_NAME, 'li')
list_month_options = len(months_list)

In [8]:
# Identifying the "Consulta" button
driver.find_element( By.ID, "ctl00_cphContent_btnConsultaMensual").click()
driver.get("https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C")

In [9]:
# We can notice that the buttons change the id with Mn or Mex if its national or foreign currency, so we generate a list that has the value of string in order to generate a loop for currency
driver.find_element( By.ID, "ctl00_cphContent_lbtnMn").click()
driver.find_element( By.ID, "ctl00_cphContent_lbtnMex").click()
list_currency = ["Mn", "Mex"]

In [10]:
# We generate two arrays for the year and month, starting with the second item of the list
year_numbers_opt = np.array( [1, 0 ] + np.arange(2, list_year_options).tolist() )
month_numbers_opt = np.array( [1, 0 ] + np.arange(2, list_month_options).tolist() )

### Downloading tables from interest rates in national and foreing currency

In [ ]:
# We generate list of years, months and tables, in order to identify the different objects that we are webscrapping
years_info_list = []
months_info_list = []
tables_info_list = []

# We open our page
driver.get("https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=C")

# Generating a for loop for every year in the dropdown list
for year_number in year_numbers_opt:

    # Dropdown button list
    dropdown_button_year = driver.find_element( By.ID, "ctl00_cphContent_rAnio")
    dropdown_button_year.click()

    # List of years options
    time.sleep(0.5)
    driver.find_element( By.XPATH,  f"/html/body/form/div[1]/div/ul/li[{year_number + 1 }]" ).click()
    year_text = driver.find_element( By.ID, "ctl00_cphContent_rAnio").text

    # Generating a for loop for every month in the dropdown list
    for month_number in month_numbers_opt:

        # Dropdown button list
        dropdown_button_month = driver.find_element( By.ID, "ctl00_cphContent_rMes")
        dropdown_button_month.click()

        # # List of months options
        time.sleep(0.5)
        driver.find_element( By.XPATH,  f"/html/body/form/div[1]/div/ul/li[{month_number + 1 }]" ).click()
        month_text = driver.find_element( By.ID, "ctl00_cphContent_rMes").text
        print( year_text )
        print( month_text )

        # Click consultar
        driver.find_element( By.ID,"ctl00_cphContent_btnConsultaMensual").click()
        time.sleep(5)

        year_float = int(year_text)

        # Generating a for loop for the two types of currencies
        for currency in list_currency:
            # Selecting the currency, there are two types of ids for the currency buttons
            try:
                driver.find_element( By.ID, f"ctl00_cphContent_lbtn{currency}" ).click()
                time.sleep(5)
                print(currency)
            except:
                print("No currency tipo 1")

            try:

                driver.find_element( By.ID, f"ctl00_cphContent_lbtn{currency}ASP" ).click()
                time.sleep(5)
                print(currency)
            except:
                print("No currency tipo 2")

            # Identifying the tables in the web page, there are three types of formats in all the period of interest
            # Type 1 table - Changing the frame and extracting a simple table
            try:
                # Switch frame
                driver.switch_to.frame(driver.find_element( By.ID,f"ctl00_cphContent_pnContenidoAnteriorMnASP"))
                section_tab_table = driver.find_element( By.XPATH,  "/html/body/form/table/tbody" )
                table_html = section_tab_table.get_attribute('outerHTML')
                table_pd = pd.read_html( table_html )

                # Cleaning table
                table_clean_1 = table_pd[1].iloc[5:, :].copy().reset_index( drop = True )

                # Changing switch
                driver.switch_to.default_content()

                print("Table Tipo 1 - Cambiar de Frame y una tabla simple")

            except:
                print("No table Tipo 1")
                table_clean_1 = "Null"

            # Type 2 - Without changing the frame and extracting a complex table
            try:
                # Get tables
                section_id_table = driver.find_element( By.ID, f"ctl00_cphContent_rpgActual{currency}_OT" )
                table_html = section_id_table.get_attribute('outerHTML')
                table_pd = pd.read_html( table_html )

                # generate table objects
                table0, table1 = table_pd[0], table_pd[1]
                table_clean_2 = table1.copy()

                # Number of rows of second table
                tb1_rows = table1.shape[0]
                drop_rows = np.arange( 1, tb1_rows + 2 )

                # Clean first table
                table0 = table0.drop( drop_rows, axis = 0 )

                # Assign values to new table
                col_name = table0.iloc[0, 0]
                col_values = table0.iloc[1:, 0].tolist()
                table_clean_2.insert( loc = 0, column= col_name, value = col_values)

                print("Table Tipo 2 - No Cambiar de Frame y una tabla Compleja")

            except:
                print("No table Tipo 2")
                table_clean_2 = "Null"


            # Check what to save
            if ( ( isinstance( table_clean_2, str ) == False ) and ( table_clean_2.shape[1] > 5 ) ):

                # Data with no frame
                table_clean_2.to_excel( fr"..\..\..\Diplomado_PUCP\_output\sbs\municipal\group5/table_clean_{year_text}_{month_text}_{currency}.xlsx", index = False )

                years_info_list.append( year_float)
                months_info_list.append( month_text )
                tables_info_list.append( "Table 2" )
                print("guardo2")

            elif ( ( isinstance( table_clean_1, str ) == False ) and ( table_clean_1.shape[1] > 5 ) ):
                # Data with frame and 1 table
                table_clean_1.to_excel( fr"..\..\..\Diplomado_PUCP\_output\sbs\municipal\group5/table_clean_{year_text}_{month_text}_{currency}.xlsx", index = False , header = False )

                years_info_list.append( year_float)
                months_info_list.append( month_text )
                tables_info_list.append( "Table 1" )
                print("guardo1")

            # No table information
            # Go to a next month/year combination
            else:
                # Day ago
                years_info_list.append( year_float)
                months_info_list.append( month_text )
                tables_info_list.append( "No Table" )

            driver. switch_to.default_content()


In [12]:
# Validation of the years, months and table types
years_unique = list(set(years_info_list))
months_unique = list(set(months_info_list))
tables_unique = list(set(tables_info_list))